# Web Scraping Project

In [2]:
# Импортируем библиотеки, которые будем использовать.
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
# На момент написания проекта в разделе "сериалы" содержится 335 страниц, каталог пополняется на постоянной основе.
# Выставляем руками количество страниц, запускаем цикл, получаем ответ от каждой страницы, парсим страничку в дерево.
# Находим теги с информацией о сериале, "забираем" ссылки на сериалы и оригинальные названия в словарь d. В список links
# добавляем словарь d

page = 1
max_page = 336
links = []
while page < max_page:
    url = f'https://doramy.club/serialy/page/{page}'
    r = requests.get(url)
    if r.status_code == 200:
        print(f'Get page {page}') # Данная строчка не обязательна, служит для визуальной проверки доступа к странице каталога. 
        soup = BeautifulSoup(r.text)

        content = soup.find('div', {'class': 'art content'})
        if content != None:
            for post in content.find_all('div', {'class': 'post-home'}):
                d = {'link': '', 'origin_title': ''}
                if post.find('a') != None:
                    d['link'] = post.find('a').get('href')
                    if post.find('em') != None:
                        d['origin_title'] = post.find('em').text
                               
                    links.append(d)
    else:
        break
    page += 1

Get page 1
Get page 2
Get page 3
Get page 4
Get page 5
Get page 6
Get page 7
Get page 8
Get page 9
Get page 10
Get page 11
Get page 12
Get page 13
Get page 14
Get page 15
Get page 16
Get page 17
Get page 18
Get page 19
Get page 20
Get page 21
Get page 22
Get page 23
Get page 24
Get page 25
Get page 26
Get page 27
Get page 28
Get page 29
Get page 30
Get page 31
Get page 32
Get page 33
Get page 34
Get page 35
Get page 36
Get page 37
Get page 38
Get page 39
Get page 40
Get page 41
Get page 42
Get page 43
Get page 44
Get page 45
Get page 46
Get page 47
Get page 48
Get page 49
Get page 50
Get page 51
Get page 52
Get page 53
Get page 54
Get page 55
Get page 56
Get page 57
Get page 58
Get page 59
Get page 60
Get page 61
Get page 62
Get page 63
Get page 64
Get page 65
Get page 66
Get page 67
Get page 68
Get page 69
Get page 70
Get page 71
Get page 72
Get page 73
Get page 74
Get page 75
Get page 76
Get page 77
Get page 78
Get page 79
Get page 80
Get page 81
Get page 82
Get page 83
Get page 84
G

Данная функция получает на вход ссылку на страницу с описанием сериала из словаря d. В случае успешного запроса парсит 
страницу в дерево, добавляет в словарь d русскоязычное название, ссылку на постер фильма, описание, рейтинг, кол-во оценок. 
Извлечение остальной информации о сериале представляет определенную сложность.
Для ее получения находим участок html разметки с этой информацией, запускаем отдельный цикл, используя вспомогательный словарь prop_names 'достаем' данные и добавляем в словарь d.

In [4]:
def parse_movie(d):
    r = requests.get(d['link'])

    prop_names = {'Просмотр:': 'view_time', 'Сериал:': 'series', 'Статус:': 'status',
                'Добавлена:': 'last_added', 'Год:': 'year', 'Страна:': 'country',
                'Жанр:': 'genre', 'Перевод:': 'translation', 
                'Озвучка:': 'transl_group', 'В ролях:': 'actors'}

# Данный цикл, используя значения словаря prop_names как ключи для словаря d, создает в словаре d ключи с пустым значением. 
# Это позволяет оставить пропуски для сериалов, перевод которых уже завершен (у которых не указан статус и добавленные серии).
    for value in prop_names.values():
        d[value] = ''

    if r.status_code == 200:
        print('Get film ' + d['link']) # Данная строчка не обязательна, служит для визуальной проверки доступа к стр-це сериала.

        soup = BeautifulSoup(r.text)
        d['rus_title'] = soup.find('h1').text
        d['img'] = soup.find('img', {'class': 's-img'}).get('src')
        d['description'] = soup.find('div', {'class': 'annotaciya'}).text.strip()
        d['rating'] = soup.find('span',{'itemprop':'ratingValue'}).text
        d['rating_count'] = soup.find('span',{'itemprop':'ratingCount'}).text

        table = soup.find('tbody', {'class': 'tbody-sin'})
        for tr in table.find_all('tr'):
            cells = tr.find_all('td')
            if cells and len(cells) > 1:
                name = cells[0].text
                value = cells[1].text
                if name in prop_names:
                    d[prop_names[name]] = value

    return d

In [5]:
# Итоговый словарь с информацией о всех серилах.
films = {}

In [6]:
# Данный цикл посредством ф-ции parse_movie формирует итоговый словарь films 
for d in links:
    if d['link'] not in films:
        films[d['link']] = parse_movie(d)

Get film https://doramy.club/30756-moya-zhizn-snova.html
Get film https://doramy.club/30823-moj-dnevnik-osvobozhdeniya.html
Get film https://doramy.club/31185-specialist-po-ustanovleniyu-otcovstva.html
Get film https://doramy.club/26340-1-vechnaya-lyubov-3.html
Get film https://doramy.club/31236-osobnyak.html
Get film https://doramy.club/31242-veter-iz-lunsi.html
Get film https://doramy.club/31450-kogda-ty-budesh-mnoj.html
Get film https://doramy.club/30950-v-poiskax-sari.html
Get film https://doramy.club/31660-podnimaetsya-burya.html
Get film https://doramy.club/27446-sekrety-v-reshyotke.html
Get film https://doramy.club/30620-roza-v-zvezdnom-more.html
Get film https://doramy.club/27850-moya-koroleva-peregovorov.html
Get film https://doramy.club/26383-s-vozvrashheniem-mone.html
Get film https://doramy.club/29222-ya-gody-ty-zvezdy.html
Get film https://doramy.club/29998-mercaet-mercaet-yarkaya-zvezda.html
Get film https://doramy.club/29798-krasavica-s-legkim-makiyazhem.html
Get film ht

In [7]:
# Получаем датафрейм.
df = pd.DataFrame()

for film in films.values():
    df = df.append(pd.Series(film), ignore_index=True)

df

,link,origin_title,view_time,series,status,last_added,year,country,genre,translation,transl_group,actors,rus_title,img,description,rating,rating_count
0,https://doramy.club/30756-moya-zhizn-snova.html,Again My Life,1 час,16 серий,,,2022,Южная Корея,"Детектив, Драма, Мистика","Русская озвучка, Русские субтитры",SoftBox,"Ким Джи Ын, Ким Чжэ Гён, Ли Гю Хан, Ли Джун Ки",Моя жизнь снова дорама (2022),https://doramy.club/wp-content/uploads/2022/03...,"Главный герой дорамы «Моя жизнь снова», Ким Хи...",9.45,1167
1,https://doramy.club/30823-moj-dnevnik-osvobozh...,My Liberation Diary,1 час,16 серий,Выходит,"14 серия - озвучка,14 серия - субтитры",2022,Южная Корея,"Драма, Мелодрама",Русская озвучка,SoftBox,"Ким Чжи Вон, Ли Ки У, Ли Мин Ки, Ли Эль, С...",Мой дневник освобождения дорама (2022),https://doramy.club/wp-content/uploads/2022/03...,В центре сюжета дорамы «Мой дневник освобожден...,9.41,323
2,https://doramy.club/31185-specialist-po-ustano...,Qin Zi Jian Ding Shi Shou Ji,30 мин,24 серии,Выходит,8 серия - субтитры,2022,Китай,Драма,Русские субтитры,,,Специалист по установлению отцовства дорама (2...,https://doramy.club/wp-content/uploads/2022/04...,"Дун Ниншу девушка с хладнокровным характером, ...",6.73,11
3,https://doramy.club/26340-1-vechnaya-lyubov-3....,Shuang Shi Chong Fei III,45 мин,30 серий,Выходит,"22 серия - озвучка,14 серия - субтитры",2021,Китай,"Исторический, Комедия, Мелодрама","Русская озвучка, Русские субтитры",,"Лян Цзе, Син Чжао Линь, Сунь И Нин, Фан И Л...",Вечная любовь 3 дорама (2021),https://doramy.club/wp-content/uploads/2021/06...,Сюжет дорамы «Вечная любовь 3» вновь разворачи...,9.42,192
4,https://doramy.club/31236-osobnyak.html,Jangmi Mansion,35 мин,12 серий,Выходит,"7 серия - озвучка,9 серия - субтитры",2022,Южная Корея,"Детектив, Триллер","Русская озвучка, Русские субтитры",,"Им Джи Ён, Ли Мун Сик, Юн Гюн Сан",Особняк дорама (2022),https://doramy.club/wp-content/uploads/2022/04...,Напряженная дорама расскажет грустную историю ...,9.42,64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3331,https://doramy.club/43-zavtra-s-toboj.html,Naeil geudaewa,1 час 10 мин,16 серий,,,2016,Южная Корея,"Комедия, Мелодрама, Фантастика",Русская озвучка,SoftBox,"Ким Вон Хэ, Ли Чжэ Хун, Шин Мин А",Завтра с тобой дорама (2016),https://doramy.club/wp-content/uploads/2017/04...,Романтическая история путешественника во време...,8.5,787
3332,https://doramy.club/39-moj-malysh.html,Mai liteul beibi,35 мин,16 серий,,,2016,Южная Корея,"Драма, Комедия","Русская озвучка, Русские субтитры",HighHopes,,Мой малыш дорама (2016),https://doramy.club/wp-content/uploads/2017/04...,В центре событий комедийной дорамы «Мой малыш»...,8.13,416
3333,https://doramy.club/25-podsudimyj.html,Defendant / Pigoin,1 час,18 серий,,,2017,Южная Корея,"Детектив, Триллер",Русская озвучка,SoftBox,"Квон Ю Ри, Ким Кён Нам, О Чхан Сок, Ом Ки Ч...",Подсудимый дорама (2017),https://doramy.club/wp-content/uploads/2017/04...,Главный герой детективного сериала «Подсудимый...,8.82,971
3334,https://doramy.club/365-1-zolushka-i-chetyre-r...,Cinderella and Four Knights,1 час,16 серий,,,2016,Южная Корея,"Драма, Комедия, Мелодрама","Русская озвучка, Русские субтитры",SoftBox,"Ан Джэ Хён, Ли Джон Шин, На Ин У, Пак Со Да...",Золушка и четыре рыцаря дорама (2016),https://doramy.club/wp-content/uploads/2017/05...,Молодая и жизнерадостная Ха Вон не смотря на т...,9.11,8691


In [8]:
# Сохраняем датафрейм
df.to_csv('DB_DoramyClub.csv', index = False)
df.to_excel('DB_DoramyClub.xlsx', index = False)
